In [1]:
import os 
from uprep import tic, toc, print_timing
from uprep import tilenames_tls,riofill 
from uprep import (merge_tile_files,match_raster_to_reference,
                   clip_raster_to_template,clip_raster_to_template_extent,
                   parallel_merge_all,Fdod,raster_calc)
from uvars import archive_dir
from glob import glob


tile12dir = '/media/ljp238/12TBWolf/BRCHIEVE/TILES12'
mosaic_dir = "/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik" #mosaic
#give the data better names
vnamelist = ['tdem_dem_egm_v_3_gap','tdem_dem_egm','edem_egm',
             'tdem_hem','esawc','esawc_x','s1', 's2', 
             'lgeoid','ldem','ldem_egm','egm08'
             ]#'tdem_dem_egm_v_3_gap_gF3_0_100_inv_dist',

# clean this 
# compress and upload  to OndeDrive / SharePoint to use in WX2 


tdem_fs = glob(f"{archive_dir}/DEMsProducts/TDEMX/*/DEM/*_DEM.tif")
edem_fs  =  glob(f"{archive_dir}/DEMsProducts/EDEMx/TILES/comprexn/*/EDEM/*_EDEM_EGM.tif")
gdtm_v_fn = f"{archive_dir}/TargetProducts/GEDI/GRID/comprexn/GEDI_L3_be/GEDI03_elev_lowestmode_mean_2019108_2022019_002_03_EPSG4326.tif"
geoid_fn = f"{archive_dir}/AUXsProducts/GEOID/GLOBAL/us_nga_egm2008_1.tif"

In [2]:
print(tilenames_tls)
os.makedirs(mosaic_dir, exist_ok=True)
print('---- Brchieve files @merge ----')
parallel_merge_all(vnamelist, tile12dir, mosaic_dir, tilenames_tls,ncpu=10)

['N11E104', 'N11E105', 'N12E103', 'N12E104', 'N12E105', 'N13E103', 'N13E104', 'N13E105']
---- Brchieve files @merge ----
Processing pattern: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/*/*tdem_dem_egm.tif, found 17 files.Processing pattern: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/*/*tdem_dem_egm_v_3_gap.tif, found 17 files.Processing pattern: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/*/*ldem.tif, found 17 files.Processing pattern: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/*/*edem_egm.tif, found 17 files.Processing pattern: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/*/*s2.tif, found 17 files.Processing pattern: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/*/*esawc_x.tif, found 17 files.Processing pattern: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/*/*tdem_hem.tif, found 17 files.Processing pattern: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/*/*s1.tif, found 17 files.Processing pattern: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/*/*esawc.tif, found 17 files.Processing pattern: /media/ljp238/12TBWolf/BRCHIEVE/TIL

In [3]:
from xSimpleGapFill import gfill_with_data,run_gdal_fillnodata
method = "inv_dist" #["nearest","inv_dist"]

In [4]:
files = glob(f'{mosaic_dir}/*.tif'); print(len(files), 'files in mosaic dir')

19 files in mosaic dir


In [5]:
# clip the other variables for the block from vrt to the extent 

In [6]:
EGM08 = [i for i in files if 'EGM08.tif' in i][0]
ESAWC = [i for i in files if 'ESAWC.tif' in i][0]
TDEM_gap = fi = [i for i in files if 'TDEM_DEM_EGM_V_3_GAP.tif' in i][0] 


f1 = f"{mosaic_dir}/{os.path.basename(fi)[:-4]}_gF1.tif" # zero
f2 = f"{mosaic_dir}/{os.path.basename(fi)[:-4]}_gF2.tif" # egm 
f3 = f"{mosaic_dir}/{os.path.basename(fi)[:-4]}_gF3.tif" # iwd

gfill_with_data(fi, fi_fill=EGM08, esa=ESAWC, fo=f2, 
                fo_mask=None, chunk_size=1024, 
                threshold=-30, nodata_out=-9999)

Output file /media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_V_3_GAP_gF2.tif already exists. Skipping processing.


'/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_V_3_GAP_gF2.tif'

In [7]:
f3fill = run_gdal_fillnodata(src_path=f2, dst_path=f3, 
                    md=100, si=0,method=method, output_format="GTiff", band=1)

Infile ... /media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_V_3_GAP_gF3.tif
Output file /media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_V_3_GAP_gF3_0_100_inv_dist.tif already exists. Skipping processing.


In [8]:
#GDEM 

In [9]:
block_geoid1k = f"{mosaic_dir}/EGM08_1Km.tif"
block_geoid12 = f"{mosaic_dir}/EGM08_12m.tif"
# tdem_wgs_a_fn>TDEM_gap

if os.path.isfile(block_geoid12):
    print(f"{block_geoid12} already exists, skipping creation.")
else:
    clip_raster_to_template(geoid_fn, TDEM_gap,block_geoid12)

if os.path.isfile(block_geoid1k):
    print(f"{block_geoid1k} already exists, skipping creation.")
else:
    clip_raster_to_template_extent(geoid_fn, TDEM_gap, block_geoid1k)
# this are not aliinged , so we need to make sure they are aligned

/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/EGM08_12m.tif already exists, skipping creation.
/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/EGM08_1Km.tif already exists, skipping creation.


In [19]:
block_gdtmv = f"{mosaic_dir}/GEDI03_vdtm_WGS_1Km.tif"
block_gdtmf = f"{mosaic_dir}/GEDI03_fdtm_WGS_1Km.tif"

# this filling only clips the raster to the template extent
if os.path.isfile(block_gdtmv):
    print(f"{block_gdtmv} already exists, skipping creation.")
else:
    clip_raster_to_template_extent(gdtm_v_fn, TDEM_gap, block_gdtmv)
    # tdem_wgs_a_fn>TDEM_gap

print('---- Filling files @Gdem ----')
if not os.path.isfile(block_gdtmf):
    print(f"Filling... {block_gdtmf}")
    riofill(block_gdtmv, block_gdtmf, si=0) # replace by IWD


print('---- Transforming Href files @Gdem ----')
block_gdtmf_egm = f"{mosaic_dir}/GEDI03_fdtm_EGM_1Km.tif"
if not os.path.isfile(block_gdtmf_egm):
    print(f"Transforming... {block_gdtmf_egm}")
    Fdod(fine_path=block_gdtmf, coarse_path=EGM08, output_path=block_gdtmf_egm) 
    # may be the function inside Ffod is missalinging them 
    # block_geoid > EGM08 this one is alighed but error due to gdem covering all 
    # block_geoid > block_geoid1k
    print(block_gdtmf_egm)

/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_vdtm_WGS_1Km.tif already exists, skipping creation.
---- Filling files @Gdem ----
---- Transforming Href files @Gdem ----


In [20]:
tdem_r = [i for i in files if 'TDEM_DEM_EGM.tif' in i][0]
# DOD : for TDEMX void and filled and EDEM

In [21]:
f3filldod = f3fill.replace('.tif', '_dod.tif')
raster_calc(f3fill, block_gdtmf_egm, 'sub', f3filldod, priority='rbpath', overwrite=False)

Output file '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_V_3_GAP_gF3_0_100_inv_dist_dod.tif' already exists. Skipping operation. Set 'overwrite=True' to force overwrite.


In [22]:
tdem_rdod = tdem_r.replace('.tif', '_dod.tif')
raster_calc(tdem_r, block_gdtmf_egm, 'sub', tdem_rdod, priority='rbpath', overwrite=False)

Prioritizing properties from raster2 for reprojection and alignment.
Reprojecting '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM.tif' to match '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM_1Km.tif'...
Reprojection of raster1 complete.
Reprojecting '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM_1Km.tif' to match '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM_1Km.tif'...
Reprojection of raster2 complete.
Performing 'sub' operation...
Operation complete. Writing results...
Operation successfully completed. Result saved to: /media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_dod.tif


In [23]:
fidod = fi.replace('.tif', '_dod.tif')
raster_calc(fi, block_gdtmf_egm, 'sub', fidod, priority='rbpath', overwrite=False)

Prioritizing properties from raster2 for reprojection and alignment.
Reprojecting '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_V_3_GAP.tif' to match '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM_1Km.tif'...
Reprojection of raster1 complete.
Reprojecting '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM_1Km.tif' to match '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM_1Km.tif'...
Reprojection of raster2 complete.
Performing 'sub' operation...
Operation complete. Writing results...
Operation successfully completed. Result saved to: /media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_V_3_GAP_dod.tif


In [ ]:



fidod = fi.replace('.tif', '_dod.tif')
raster_calc(fi, block_gdtmf_egm, 'sub', fidod, priority='rbpath', overwrite=False)
fi
tdem_r

block_gdtmf_egm = f"{mosaic_dir}/GEDI03_fdtm_EGM.tif"
f3filldod = f3fill.replace('.tif', '_dod.tif')
raster_calc(f3fill, block_gdtmf_egm, 'sub', f3filldod, priority='rbpath', overwrite=False)







Output file '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_V_3_GAP_gF3_0_100_inv_dist_dod.tif' already exists. Skipping operation. Set 'overwrite=True' to force overwrite.
Error reading or writing raster file: /media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM.tif: No such file or directory
Error reading or writing raster file: /media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM.tif: No such file or directory


In [13]:
f3dod = f3fill.replace('.tif', '_dod.tif')
raster_calc(f3fill, block_gdtmf_egm, 'sub', f3dod, priority='rbpath', overwrite=False)

Prioritizing properties from raster2 for reprojection and alignment.
Reprojecting '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_V_3_GAP_gF3_0_100_inv_dist.tif' to match '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM_1Km.tif'...
Reprojection of raster1 complete.
Reprojecting '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM_1Km.tif' to match '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/GEDI03_fdtm_EGM_1Km.tif'...
Reprojection of raster2 complete.
Performing 'sub' operation...
Operation complete. Writing results...
Operation successfully completed. Result saved to: /media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/TLS/mosaik/TDEM_DEM_EGM_V_3_GAP_gF3_0_100_inv_dist_dod.tif
